In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.callbacks import get_openai_callback

These are the texts in which the QA will be made

In [2]:
texts = ["the dog of San Roque does not have a tail", 
         "Don Gato was a cat, a cat of high society, always walked with elegance and great discretion",
         "Alonso has won 32 races and 2 world championships",
         "Real Betis is the biggest team in the world"
         ]

These are the queries

In [3]:
queries = ["Who doesn´t have tail?", "¿quién estaba sentadito en su tejado?", "Tell me about Alonso", "biggest team?"]

Here an embbeded vector database is created with the documents.

In [4]:
docsearch = Chroma.from_texts(texts, OpenAIEmbeddings(), metadatas=[{"source": str(i)} for i in range(len(texts))])


In [5]:
# first it looks for the most similar documents and then it passes those documents to chatgpt and ask the question again.
with get_openai_callback() as cb:
    for query in queries:
        print(query)
        docs = docsearch.similarity_search(query) 
        chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", verbose=True)
        print(chain({"input_documents": docs, "question": query}, return_only_outputs=True))
        print("==========================================")

    print(cb) # quite expensive because it sends a lot of tokens to "educate" chatgpt

Who doesn´t have tail?


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.

11.7 Severability. The invalidity, ill